[View in Colaboratory](https://colab.research.google.com/github/norikaisa/MIMIC_eICU/blob/master/NLP%20-%20Part%20C%20-%20Pneumothorax.ipynb)

# Pneumothorax example

## Sentence tokenization, and spotting term + negation

This example spots meantions of the "pneumothorax" lexicon in CXR reports and looks at whether the spotted pneumothorax mentioned was negated or not. 

*Joy Wu* <joy.wu@ibm.com>*, *Daniel Gruhl <dgruhl@us.ibm.com>*

In [0]:
# Required files
import requests
from requests.auth import HTTPBasicAuth
import json
import matplotlib.pyplot as plt
import subprocess
import tempfile
import os
import sys
import pandas as pd
import numpy as np
import re
import random
from nltk.tokenize import sent_tokenize



from google.colab import auth
from google.cloud import bigquery
from google.colab import files

**Authenticate:** The line of code below ensures you are an authenticated user accessing the MIMIC database. You will need to rerun this each time you open the notebook.

In [0]:
auth.authenticate_user() #This will allow you to authenticate access to BigQuery

**Query Function: **This is a method that executes a desired SQL query on the database. If you want to run a query, you can use the function name below, which we named run_query()


In [0]:
project_id='hst-953-2018'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

### Sentence splitting:

In [3]:
# Read the sample CXR reports into a pandas dataframe, and print out a random report
#CXRreports = pd.read_csv('mimic3_1000cxrReports.csv')
#CXRreports.head()


CXRreports = run_query('''
SELECT * 
FROM `hst-953-2018.NLP_workshop.cxr`
''')
CXRreports.head()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=wXpnzx97A83XVebTu6IBw78jrTw3U4&prompt=consent&access_type=offline
Enter the authorization code: 4/dgB6JMxLVND4cErqjYpmoN749zfretTupxmQCwAAXVZvTQaVvMOw3Os


,subject_id,hadm_id,row_id,text
0,256,155415,739694,[**2163-7-27**] 11:44 AM\n CHEST (PORTABLE AP)...
1,13316,172688,738814,[**2104-2-19**] 4:20 PM\n CHEST (SINGLE VIEW) ...
2,5637,157276,738831,[**2194-3-20**] 11:32 AM\n CHEST (PORTABLE AP)...
3,22533,145806,741019,[**2182-7-16**] 2:44 PM\n CHEST (PORTABLE AP) ...
4,1287,111856,740821,[**2138-7-26**] 6:49 AM\n CHEST (PORTABLE AP) ...


In [4]:
#This prints a random report
report = CXRreports.text[random.randint(0,1000)]
print(report)

[**2176-7-10**] 5:34 AM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 29270**]
 Reason: eval for CHF/ pulmonary edema
 ______________________________________________________________________________
 [**Hospital 4**] MEDICAL CONDITION:
  83 year old woman with infected pseudocyst in pancreas, now altered mental
  status, bilateral crackles new at bases.
 REASON FOR THIS EXAMINATION:
  eval for CHF/ pulmonary edema
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION:  Rectal pseudocyst. Altered mental status.

 SINGLE VIEW CHEST:   Comparison is made to prior radiograph of [**2176-7-7**].

 A central venous line catheter is in the upper SVC.  There has been interval
 placement of an NG tube which is seen to course to the stomach.  Again
 demonstrated is a calcified plaque in the left lateral hemithorax.  There is
 bilateral basilar atelectasis and a r

In [5]:
  #This imports nltk and punkt into our environment
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Tokenize the sentences with sent_tokenize from NLTK
from nltk.tokenize.punkt import PunktSentenceTokenizer
# Alternatively, tokenize with PunktSentenceTokenizer from NLTK if you want to keep track of character offsets of sentences
sents = sent_tokenize(report.replace('\n',' ')) # removing new line breaks
# Print out list of sentences:
sent_count = 0
for s in sents:
    print("Sentence " + str(sent_count) +":")
    print(s)
    print()
    sent_count = sent_count + 1

Sentence 0:
[**2176-7-10**] 5:34 AM  CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 29270**]  Reason: eval for CHF/ pulmonary edema  ______________________________________________________________________________  [**Hospital 4**] MEDICAL CONDITION:   83 year old woman with infected pseudocyst in pancreas, now altered mental   status, bilateral crackles new at bases.

Sentence 1:
REASON FOR THIS EXAMINATION:   eval for CHF/ pulmonary edema  ______________________________________________________________________________                                  FINAL REPORT  INDICATION:  Rectal pseudocyst.

Sentence 2:
Altered mental status.

Sentence 3:
SINGLE VIEW CHEST:   Comparison is made to prior radiograph of [**2176-7-7**].

Sentence 4:
A central venous line catheter is in the upper SVC.

Sentence 5:
There has been interval  placement of an NG tube which is seen to course to the stomach.

Sentence 6:
Again  demonstrated is a calcified plaq

In [7]:
sent_count = 0
for s_start, s_finish in PunktSentenceTokenizer().span_tokenize(report):
    print("Sentence " + str(sent_count) +": " + str([s_start, s_finish]))
    print(report[s_start:s_finish].replace('\n',' '))
    print()
    sent_count = sent_count + 1

Sentence 0: [0, 407]
[**2176-7-10**] 5:34 AM  CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 29270**]  Reason: eval for CHF/ pulmonary edema  ______________________________________________________________________________  [**Hospital 4**] MEDICAL CONDITION:   83 year old woman with infected pseudocyst in pancreas, now altered mental   status, bilateral crackles new at bases.

Sentence 1: [409, 628]
REASON FOR THIS EXAMINATION:   eval for CHF/ pulmonary edema  ______________________________________________________________________________                                  FINAL REPORT  INDICATION:  Rectal pseudocyst.

Sentence 2: [629, 651]
Altered mental status.

Sentence 3: [654, 732]
SINGLE VIEW CHEST:   Comparison is made to prior radiograph of [**2176-7-7**].

Sentence 4: [735, 786]
A central venous line catheter is in the upper SVC.

Sentence 5: [788, 876]
There has been interval  placement of an NG tube which is seen to course to t

### Spot occurrence(s) of word(s) related to your concept in a sentence or document

In [0]:
# Simple spotter: Spot occurrence of a term in a given lexicon anywhere within a text document or sentence:
def spotter(text, lexicon):
    text = text.lower()
    # Spot if a document mentions any of the terms in the lexicon (not worrying about negation detection yet)
    match = [x in text for x in lexicon]
    if any(match) == True:
        mentioned = 1
    else:
        mentioned = 0
    return mentioned

In [0]:
# Where the lexicon is a list of word(s) or phrase(s) refering to a concept of interest to you, e.g.
ptx = ['pneumothorax', 'ptx']
sent1 = 'Large left apical ptx present.'
sent2 = 'Hello world for NLP'


In [10]:
# lexicon mentioned in text, spotter return 1 (yes)
spotter(sent1, ptx)

1

In [11]:
# lexicon not mentioned in text, spotter return 0 (no)
spotter(sent2, ptx)

0

**How can we do better?**
We can do the spotting of concepts (lexicons) A LOT better (more sensitive) if we curate a list of all the ways that the concept could be expressed in raw text. This is what the NLP tool can help you achieve.

### Download a lexicon from the NLP tool:

In [0]:
import getpass
# Enter your team's username between the quotation marks:
user = "team1"
# Enter your team's password
#password = getpass.getpass()
# If the above doesn't work, then comment out the password variable above and hard code your team's password below:
password = '(send reforms capture  teams password)'

In [0]:
# This is the id of the lexicon - you can see it in the URL line when you are working with the lexicon
# For example, for pneumothorax, it is:
oid = ".2.48"
# You can do this in a loop to download all relevant lexicons into a data format you prefer too

In [0]:
# Don't spam with insecure warnings - some machines do not have all signing authority
# root certificates preinstalled
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [0]:
# The endpoints for the REST
host = "https://dla.sl.res.ibm.com"
lexurl = host + "/oid" + oid.replace('.', '/')
quartermaster =  host + "/search"

In [16]:
# Set up auth and get the lexicon. Then pull the terms out and lower case them
auth=(user,password)
lex = requests.get(lexurl, verify=False, auth=auth).json()
terms = list(map(lambda x: x["surfaceForm"].lower(), lex["members"]))

JSONDecodeError: ignored

In [17]:
# Printing out the pneumothorax lexicon (after 5 minutes of curating work on the NLP tool)
ptx = terms.copy()
print(ptx)

NameError: ignored

### Negation detection

In [18]:
# But it's not enough to just spot word occurrences to determine if a concept is affirmative (positive/present) or not.

# e.g. lexicon mentioned in text but negated, a simple spotter would still return 1 (yes)
sent3 = 'Pneumothorax has resolved.'
spotter(sent3, ptx)

1

In [19]:
# However, if negation related words occur in close proximity (e.g. same sentence) to a spotted concept 
# Then we can right some rules to determine if the concept was negated or not

# e.g. spotting negation words in the same sentence:
neg = ['no','never','not','removed', 'ruled out', 'resolved']
spotter(sent3, neg)

1

### Using off-the-shelf python library for negation, e.g. Negex

In [20]:
#This downloads a copy of negex.py and negex_triggers.txt into this environment, we will learn how to use this in the next block
!wget  https://stuff.mit.edu/~cwc76/hst953/negex.py
!wget  https://stuff.mit.edu/~cwc76/hst953/negex_triggers.txt
  


--2018-10-12 15:57:12--  https://stuff.mit.edu/~cwc76/hst953/negex.py
Resolving stuff.mit.edu (stuff.mit.edu)... 18.4.60.31
Connecting to stuff.mit.edu (stuff.mit.edu)|18.4.60.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8984 (8.8K) [text/x-python]
Saving to: ‘negex.py’

negex.py            100%[===================>]   8.77K  --.-KB/s    in 0s      

2018-10-12 15:57:12 (137 MB/s) - ‘negex.py’ saved [8984/8984]

--2018-10-12 15:57:13--  https://stuff.mit.edu/~cwc76/hst953/negex_triggers.txt
Resolving stuff.mit.edu (stuff.mit.edu)... 18.4.60.31
Connecting to stuff.mit.edu (stuff.mit.edu)|18.4.60.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7363 (7.2K) [text/plain]
Saving to: ‘negex_triggers.txt’

negex_triggers.txt  100%[===================>]   7.19K  --.-KB/s    in 0s      

2018-10-12 15:57:14 (224 MB/s) - ‘negex_triggers.txt’ saved [7363/7363]



In [21]:
import negex
rfile = open(r'negex_triggers.txt')
irules = negex.sortRules(rfile.readlines())
rfile.close()

# Example:
sent = "There is no evidence of ptx."
ptx = ['pneumothorax', 'ptx']
tagger = negex.negTagger(sentence = sent, phrases = ptx, rules = irules, negP=False)
negation = tagger.getNegationFlag()
negation

'negated'

In [22]:
# Applying Negex to the first note:
# Specify the lexicon of interest ("phrases" for Negex)
ptx = terms.copy()
# Get a randome note from the dataset:
note = CXRreports['text'][random.randint(0,1000)]
# Tokenize the sentences:
sents = sent_tokenize(note.replace('\n',' ')) # replacing new line breaks
# Applying spotter function to each sentence:
#neg_output = []
count = 0
for sent in sents:
    # Apply Negex if a term in the ptx lexicon is spotted
    if spotter(sent,ptx) == 1:
        tagger = negex.negTagger(sentence = sent, phrases = ptx, rules = irules, negP=False)
        negation = tagger.getNegationFlag()
        #neg_output.append(negation)
        print("Sentence " + str(count) + ":\n" + sent + "\nNegex output: " + negation + '\n')
        count = count + 1

NameError: ignored

In [0]:
# Show the relevant CXR report for the analysis
print(note)

### Exercise for you:

You can use similar/improved pipeline to loop through all the notes in your dataset and through different concepts/lexicons!